# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Jacopo De Stefani - __[Jacopo.De.Stefani@ulb.ac.be](mailto:Jacopo.De.Stefani@ulb.ac.be)__
### Bertrand Lebichot - __[Bertrand.Lebichot@ulb.ac.be](mailto:Bertrand.Lebichot@ulb.ac.be)__
### Arnaud Pollaris - __[Arnaud.Pollaris@ulb.ac.be](mailto:Arnaud.Pollaris@ulb.ac.be)__

## TP 6 -  Ensembles of models and feature selection

####  April 30, 2019

#### Materials originally developed by *Yann-Aël Le Borgne, Fabrizio Carcillo and Gianluca Bontempi*

## Overview

Ensembles of models and feature selection are two machine learning techniques which can be used to improve the accuracy of preditions. 

Ensembles of models consist in building several predictive models using resampled subsets of the original training set. The method works particularly well for predictive models with high variance (for example, decision trees or neural networks). The average prediction of the resulting models usually strongly decreases the variance component of the error, and as a consequence improves the prediction accuracy. 

Feature selection aims at reducing the dimensionality of the problem, and is useful when input variables contain redundant or irrelevant (noisy) information. Benefits are twofold: it decreases the training time by simplifying the problem, and it decreases the complexity of the predictive model. This in turn usually improves the prediction accuracy, since high-dimensionality makes predictive models more prone to overfitting, and estimates of parameters more variant. 

In this session, we will illustrate both techniques using the IMDB 5000 dataset, which contains 27 variables describing 5043 movies. The variables contain information about the director, actors, number of Facebook likes for each actor, duration, genre, language, country, etc... We will use them to predict the movie success (through the IMDB score). The dataset together with a description of the variables is at https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset.

The dataset is on the github of the course, in datasets/movie_metadata.csv

## Data overview and preprocessing

Let us load and select a random subset of 1000 movies

In [ ]:
data<-read.csv("movie_metadata.csv")
set.seed(2)
data<-data[sample(nrow(data),1000),]

### Dataset overview

In [ ]:
dim(data)


In [ ]:
options(repr.matrix.max.cols=50)
data[1:2,]

In [ ]:
summary(data)

We see there is a mix of categorical and numerical variables, and some missing values. In order to simplify the analysis, let us remove the categorical variables, and replace the NA values with the mean values of the variables.

### Remove categorical variables

Check the type of input variables

In [ ]:
sapply(data[1,],class)

Get indices of categorical (factor) variables

In [ ]:
factor_variables<-which(sapply(data[1,],class)=="factor")
factor_variables

Remove categorical variables

In [ ]:
data_preprocessed<-data[,-factor_variables]
summary(data_preprocessed)

### Replace NA values with mean values

In [ ]:
replace_na_with_mean_value<-function(vec) {
    mean_vec<-mean(vec,na.rm=T)
    vec[is.na(vec)]<-mean_vec
    vec
}

In [ ]:
data_preprocessed<-data.frame(apply(data_preprocessed,2,replace_na_with_mean_value))
summary(data_preprocessed)

### Input and output variables

The output variable (Y) is the `imdb_score`, and all other variables (X) are considered as inputs.

In [ ]:
set.seed(3)

X<-data_preprocessed[,setdiff(colnames(data_preprocessed),"imdb_score")]
Y<-data_preprocessed[,"imdb_score"]

N<-nrow(X)    #Number of examples
n<-ncol(X)    #Number of input variables


Distribution of the `imdb_score`

In [ ]:
hist(Y)

In [ ]:
mean(Y)

In [ ]:
var(Y)

### 1) Modelling with linear and decision tree models

#### Linear model

* Let us create a linear model for predicting the IMDB score on the basis of the other variables, and compute its empricial mean square error

In [ ]:
DS<-cbind(X,imdb_score=Y)
    
model<- ### Fill with your code here
        
Y.hat<- predict(model,X)
        
empirical_error<- ### Fill with your code here

print(paste("Empirical error=",round(empirical_error,digits=4)))



* Which input variables are statistically correlated with the output?

In [ ]:
summary(model)

* Compute the validation error with a 10-fold cross-validation

In [ ]:
size.CV<-floor(N/10)

CV.err<-numeric(10)

for (i in 1:10) {
     i.ts<-  ### Complete the code. i.ts should be the indices of the tessefor the i-th fold
     X.ts<-X[i.ts,]  
     Y.ts<-Y[i.ts]  
     
     i.tr<-  ### Complete the code. i.tr should be the indices of the training sefor the i-th fold
     X.tr<-X[i.tr,]
     Y.tr<-Y[i.tr]                          
     
     DS<-cbind(X.tr,imdb_score=Y.tr)
    
     model<- lm(imdb_score~.,DS)
        
     Y.hat.ts<- predict(model,X.ts)
        
     CV.err[i]<-mean((Y.hat.ts-Y.ts)^2)
}
    

print(paste("CV error=",round(mean(CV.err),digits=4), " ; std dev=",round(sd(CV.err),digits=4)))



#### Decision tree

* Modify the previous code to compute the empirical error using a decision tree model. Use the rpart package (see `?rpart` for help)

In [ ]:
library(rpart)       ### Run install.packages("rpart") to install

In [ ]:
DS<-cbind(X,imdb_score=Y)

model<- ### Fill with you code here
        
Y.hat<- predict(model,X)
        
empirical_error<-mean((Y.hat-Y)^2) 

print(paste("Empirical error=",round(empirical_error,digits=4)))



* Plot the resulting tree using the `prp` function from the library `rpart.plot`

In [ ]:
library(rpart.plot)  ### Run install.packages("rpart.plot") to install

In [ ]:
prp(model)

* What is the 10-fold cross-validation error using a decision tree model?

In [ ]:
size.CV<-floor(N/10)

CV.err<-numeric(10)

for (i in 1:10) {
     i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
     X.ts<-X[i.ts,]  
     Y.ts<-Y[i.ts]  
     
     i.tr<-setdiff(1:N,i.ts)                
     X.tr<-X[i.tr,]
     Y.tr<-Y[i.tr]                          
     
     DS<-cbind(X.tr,imdb_score=Y.tr)
    
     model<- rpart(imdb_score~.,DS)
        
     Y.hat.ts<- predict(model,X.ts)
        
     CV.err[i]<-mean((Y.hat.ts-Y.ts)^2)
    }
    

print(paste("CV error=",round(mean(CV.err),digits=4), " ; std dev=",round(sd(CV.err),digits=4)))



## 2) Ensemble of models

Let us now create an ensemble of R linear models to make predictions. Complete the code below so that

* The training set is resampled before building a model
* The predictions of all model are averaged before testing

In [ ]:
size.CV<-floor(N/10)
R<-20

CV.err<-numeric(10)

for (i in 1:10) {
     i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
     X.ts<-X[i.ts,]  
     Y.ts<-Y[i.ts]  
     
     
     i.tr<-setdiff(1:N,i.ts)                
    
     Y.hat.ts.R<-matrix(0,nrow=nrow(X.ts),ncol=R)
    
     for (r in 1:R) {
         i.tr.resample<-    ### Complete code: Resample training set with replacement
         X.tr<-X[i.tr.resample,]
         Y.tr<-Y[i.tr.resample]                          
     
         DS<-cbind(X.tr,imdb_score=Y.tr)
    
         model<- lm(imdb_score~.,DS)
        
         Y.hat.ts.R[,r]<- predict(model,X.ts)
     
     }
    
     Y.hat.ts<-apply(Y.hat.ts.R,1,mean)
     CV.err[i]<-mean((Y.hat.ts-Y.ts)^2)
     }

print(paste("CV error=",round(mean(CV.err),digits=4), " ; std dev=",round(sd(CV.err),digits=4)))


* Is the CV error lower than with a single linear model?
* Use a decision tree as the base model. Is the CV error lower?

## 3) Feature selection

Two are the main approaches to feature selection:


* **Filter methods:** they are preprocessing methods. They attempt to
assess the merits of features from the data, ignoring the effects of
the selected feature subset on the performance of the learning
algorithm. Examples are methods that select variables by ranking them
through compression techniques (like PCA), or by computing correlation or a more advanced similarity measure such as minimum redundancy maximum relevance (mRMR) with the output.

*  **Wrapper methods:** these methods assess subsets of variables
according to their usefulness to a given predictor. The method
conducts a search for a good subset using the learning algorithm
itself as part of the evaluation function. The problem boils 
down to a problem of stochastic state space search. Example
are the stepwise methods proposed in linear regression analysis.



### Filter methods

#### Correlation with the output

* The following code performs features selection by keeping the most correlated variables with the output. Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?


In [ ]:
size.CV<-floor(N/10)

CV.err<-matrix(0,nrow=n,ncol=10)

for (i in 1:10) {
    i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
    X.ts<-X[i.ts,]  
    Y.ts<-Y[i.ts]  
     
    i.tr<-setdiff(1:N,i.ts)
    X.tr<-X[i.tr,]
    Y.tr<-Y[i.tr]
     
    correlation<-abs(cor(X.tr,Y.tr))
    ranking<-sort(correlation,dec=T,index.return=T)$ix
     
    for (nb_features in 1:n) {
        DS<-cbind(X.tr[,ranking[1:nb_features],drop=F],imdb_score=Y.tr)
        model<- lm(imdb_score~.,DS)
        
        Y.hat.ts<- predict(model,X.ts[,ranking[1:nb_features],drop=F])
        
        CV.err[nb_features,i]<-mean((Y.hat.ts-Y.ts)^2)
    }
}  

print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV.err,1,mean),digits=4), " ; std dev=",round(apply(CV.err,1,sd),digits=4)))



#### mRMR

* The following code performs features selection by using the mRMR approach (see slides 49-52 of the course http://uv.ulb.ac.be/pluginfile.php/874401/mod_resource/content/2/fsel.pdf). Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?

In [ ]:
size.CV<-floor(N/10)

CV.err<-matrix(0,nrow=n,ncol=10)

for (i in 1:10) {
    i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
    X.ts<-X[i.ts,]  
    Y.ts<-Y[i.ts]  
     
    i.tr<-setdiff(1:N,i.ts)
    X.tr<-X[i.tr,]
    Y.tr<-Y[i.tr]
    
    
    correlation<-abs(cor(X.tr,Y.tr))
    
    selected<-c()
    candidates<-1:n
    
    #mRMR ranks the variables by taking account not only the correlation with the output, but also by avoiding redudant variables
    for (j in 1:n) {
        redudancy.score<-numeric(length(candidates))
        if (length(selected)>0) {
            cor.selected.candidates<-cor(X.tr[,selected,drop=F],X.tr[,candidates,drop=F])
            redudancy.score<-apply(cor.selected.candidates,2,mean)
        }
        
        mRMR.score<-correlation[candidates]-redudancy.score
        
        selected_current<-candidates[which.max(mRMR.score)]
        selected<-c(selected,selected_current)
        candidates<-setdiff(candidates,selected_current)
    }
    
    ranking<-selected
     
    for (nb_features in 1:n) {
        DS<-cbind(X.tr[,ranking[1:nb_features],drop=F],imdb_score=Y.tr)
        model<- lm(imdb_score~.,DS)
        
        Y.hat.ts<- predict(model,X.ts[,ranking[1:nb_features],drop=F])
        
        CV.err[nb_features,i]<-mean((Y.hat.ts-Y.ts)^2)
    }
}  

print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV.err,1,mean),digits=4), " ; std dev=",round(apply(CV.err,1,sd),digits=4)))


#### PCA

* The following code performs features selection by first transforming the inputs using PCA, and then keeping the most relevant principal components in the model. Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?



In [ ]:
size.CV<-floor(N/10)

CV.err<-matrix(0,nrow=n,ncol=10)

X_pca<-data.frame(prcomp(X,retx=T)$x)

for (i in 1:10) {
    i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
    X.ts<-X_pca[i.ts,]  
    Y.ts<-Y[i.ts]  
     
    i.tr<-setdiff(1:N,i.ts)
    X.tr<-X_pca[i.tr,]
    Y.tr<-Y[i.tr]
     
    for (nb_features in 1:n) {
        DS<-cbind(X.tr[,1:nb_features,drop=F],imdb_score=Y.tr)
        model<- lm(imdb_score~.,DS)
        
        Y.hat.ts<- predict(model,X.ts[,1:nb_features,drop=F])
        
        CV.err[nb_features,i]<-mean((Y.hat.ts-Y.ts)^2)
    }
}  

print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV.err,1,mean),digits=4), " ; std dev=",round(apply(CV.err,1,sd),digits=4)))


### Wrapper method: Forward selection

* The following code performs features selection by using a forward selection method (See slide 20 in http://uv.ulb.ac.be/pluginfile.php/874401/mod_resource/content/1/fsel.pdf). Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?


In [ ]:
size.CV<-floor(N/10)

selected<-NULL

for (round in 1:n) { 
    candidates<-setdiff(1:n,selected)
    
    CV.err<-matrix(0,nrow=length(candidates),ncol=10)
    
    for (j in 1:length(candidates)) {
        features_to_include<-c(selected,candidates[j])
        
        for (i in 1:10) {
            i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
            X.ts<-X[i.ts,features_to_include,drop=F]  
            Y.ts<-Y[i.ts]  
     
            i.tr<-setdiff(1:N,i.ts)
            X.tr<-X[i.tr,features_to_include,drop=F]
            Y.tr<-Y[i.tr]
     
            DS<-cbind(X.tr,imdb_score=Y.tr)
            model<- lm(imdb_score~.,DS)
        
            Y.hat.ts<- predict(model,X.ts)
        
            CV.err[j,i]<-mean((Y.hat.ts-Y.ts)^2)
        }
    }
    CV.err.mean<-apply(CV.err,1,mean)
    CV.err.sd<-apply(CV.err,1,sd)
    selected_current<-which.min(CV.err.mean)              
    selected<-c(selected,candidates[selected_current])
    print(paste("Round ",round," ; Selected feature: ",candidates[selected_current]," ; CV error=",round(CV.err.mean[selected_current],digits=4), " ; std dev=",round(CV.err.sd[selected_current],digits=4)))

}
                   


## Further preprocessing to add categorical variables

Categorical variables usually need to be transformed with 'one-hot-encoding' in order to be processed by a learning algorithm. That is, for each value of the categorical variable, a binary feature is created, which is set to one whenever that value is present. This can be done using the `dummy.data.frame` of the `dummies` package.

```
install.packages('dummies')
library(dummies)
```

In the following, we add some categorical variables to the peprocessing dataset. The set of categorical variables is

In [ ]:
factor_variables

Let us have an overview of the their content

In [ ]:
data_factor<-data[,factor_variables]

In [ ]:
dim(data_factor)

In [ ]:
data_factor[1:2,]

Let us keep four of them: Color, language, country and content_rating, and transform them with one-hot-encoding

In [ ]:
variable_to_keep<-c("color","language","country","content_rating")

In [ ]:
data_factor_onehot <- dummy.data.frame(data_factor[,variable_to_keep], sep="_")

In [ ]:
dim(data_factor_onehot)

In [ ]:
data_factor_onehot[1:2,]

These could be added to the previously preprocessed dataset, and used to further improve the prediction accuracy using the feature selection/ensemble techniques seen above. 

In [ ]:
data_preprocessed_extended<-cbind(data_preprocessed,data_factor_onehot)

In [ ]:
dim(data_preprocessed_extended)

In [ ]:
summary(data_preprocessed_extended)

## Using other predictive models

Other models could be used, for example support vector machines, neural networks, K-nearest neighbors (using the `svm`, `nnt`or `lazy` functions from the `e1071`, `nnet` or `lazy` packages, respectively). Note that scaling the data is usually necessary when using neural networks and K-nearest neighbors approaches. 